<a href="https://colab.research.google.com/github/gmauricio-toledo/NLP-MCD/blob/main/04-N-gramas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelos de lenguaje: $N$-gramas

## Ejercicios - Tarea 2

⭕ Repetir el experimento con 2-gramas, 4-gramas y 2-skip-2-gramas.

In [1]:
# Importar librerías necesarias
import nltk
from nltk import bigrams, trigrams
from nltk.corpus import reuters
from collections import defaultdict, Counter
from itertools import chain, islice
import string

# Descargar los recursos que necesita NLTK
nltk.download('reuters')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\jcbar\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jcbar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jcbar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Función para crear el modelo de n-gramas deseado
def generate_ngrams(corpus, n, skip=0, remove_punctuation=False):
    """
    Genera n-gramas (o skip-gramas si skip > 0) a partir de un corpus.
    
    corpus: Lista de documentos
    n: Número de palabras en el n-grama
    skip: Número de palabras a saltar (solo para skip-gramas)
    remove_punctuation: Si se desean eliminar los signos de puntuación
    
    return: Lista de n-gramas
    """
    ngrams = []
    for doc in corpus:
        # Tokenización
        tokens = nltk.word_tokenize(doc.lower())
        
        # Remover signos de puntuación
        if remove_punctuation:
            tokens = [word for word in tokens if word not in string.punctuation]
        
        # Generar skip-gramas
        if skip > 0:
            tokens = list(chain(*[tokens[i::skip+1] for i in range(skip+1)]))
        
        # Crear n-gramas
        n_grams = list(nltk.ngrams(tokens, n))
        ngrams.extend(n_grams)
    
    return ngrams

En cada uno de ellos:

* **Generar el modelo de $n$-gramas (con y sin signos de puntuación).**

In [3]:
# Documentos de entrenamiento del corpus
docs = [reuters.raw(fileid) for fileid in reuters.fileids()]

# Generar 2-gramas con y sin signos de puntuación
bigrams_with_punctuation = generate_ngrams(docs, 2, remove_punctuation=False)
bigrams_without_punctuation = generate_ngrams(docs, 2, remove_punctuation=True)

print("Ejemplo de 2-gramas con signos de puntuación:")
print(bigrams_with_punctuation[:5])

print("\nEjemplo de 2-gramas sin signos de puntuación:")
print(bigrams_without_punctuation[:5])

Ejemplo de 2-gramas con signos de puntuación:
[('asian', 'exporters'), ('exporters', 'fear'), ('fear', 'damage'), ('damage', 'from'), ('from', 'u.s.-japan')]

Ejemplo de 2-gramas sin signos de puntuación:
[('asian', 'exporters'), ('exporters', 'fear'), ('fear', 'damage'), ('damage', 'from'), ('from', 'u.s.-japan')]


In [4]:
# Generar 4-gramas con y sin signos de puntuación
fourgrams_with_punctuation = generate_ngrams(docs, 4, remove_punctuation=False)
fourgrams_without_punctuation = generate_ngrams(docs, 4, remove_punctuation=True)

print("\nEjemplo de 4-gramas con signos de puntuación:")
print(fourgrams_with_punctuation[:5])

print("\nEjemplo de 4-gramas sin signos de puntuación:")
print(fourgrams_without_punctuation[:5])


Ejemplo de 4-gramas con signos de puntuación:
[('asian', 'exporters', 'fear', 'damage'), ('exporters', 'fear', 'damage', 'from'), ('fear', 'damage', 'from', 'u.s.-japan'), ('damage', 'from', 'u.s.-japan', 'rift'), ('from', 'u.s.-japan', 'rift', 'mounting')]

Ejemplo de 4-gramas sin signos de puntuación:
[('asian', 'exporters', 'fear', 'damage'), ('exporters', 'fear', 'damage', 'from'), ('fear', 'damage', 'from', 'u.s.-japan'), ('damage', 'from', 'u.s.-japan', 'rift'), ('from', 'u.s.-japan', 'rift', 'mounting')]


In [5]:
# Generar 2-skip-2-gramas con y sin signos de puntuación
skip_2_gram_with_punctuation = generate_ngrams(docs, 2, skip=2, remove_punctuation=False)
skip_2_gram_without_punctuation = generate_ngrams(docs, 2, skip=2, remove_punctuation=True)

print("\nEjemplo de 2-skip-2-gramas con signos de puntuación:")
print(skip_2_gram_with_punctuation[:5])

print("\nEjemplo de 2-skip-2-gramas sin signos de puntuación:")
print(skip_2_gram_without_punctuation[:5])



Ejemplo de 2-skip-2-gramas con signos de puntuación:
[('asian', 'damage'), ('damage', 'rift'), ('rift', 'friction'), ('friction', 'u.s.'), ('u.s.', 'has')]

Ejemplo de 2-skip-2-gramas sin signos de puntuación:
[('asian', 'damage'), ('damage', 'rift'), ('rift', 'friction'), ('friction', 'u.s.'), ('u.s.', 'has')]


* **Realizar el experimento de predecir la siguiente palabra con el texto. Evaluar el experimento.**

In [6]:
# Función para entrenar un modelo de n-gramas
def train_ngram_model(ngrams):
    """
    Entrena un modelo basado en la frecuencia de los n-gramas.
    
    ngrams: Lista de n-gramas
    
    return: Un diccionario que mapea los (n-1)-gramas a cada posible palabra siguiente con sus frecuencias respectivas
    """
    
    # Instanciamos defaultdict con contador para almacener el número de veces que aparece cada n-grama
    model = defaultdict(Counter)
    for ngram in ngrams:
        prefix, word = tuple(ngram[:-1]), ngram[-1]
        model[prefix][word] += 1
    return model


In [7]:
# Entrenar el modelo 2-gramas con y sin signos de puntuación
bigram_model_with_punctuation = train_ngram_model(bigrams_with_punctuation)
bigram_model_without_punctuation = train_ngram_model(bigrams_without_punctuation)

# Entrenar el modelo 4-gramas con y sin signos de puntuación
fourgram_model_with_punctuation = train_ngram_model(fourgrams_with_punctuation)
fourgram_model_without_punctuation = train_ngram_model(fourgrams_without_punctuation)

# Entrenar el modelo 2-skip-2-gramas con y sin signos de puntuación
skip_2_gram_model_with_punctuation = train_ngram_model(skip_2_gram_with_punctuation)
skip_2_gram_model_without_punctuation = train_ngram_model(skip_2_gram_without_punctuation)


In [8]:
# Función para predecir la siguiente palabra con base en un (n-1)-grama
def predict_next_word(model, text, n):
    """
    Predice la siguiente palabra en una secuencia dada utilizando un modelo n-grama.
    
    model: Un modelo n-grama entrenado
    text: La secuencia de texto (lista de palabras)
    n: El tamaño del n-grama
    
    return: La palabra más probable para continuar la secuencia
    """
    
    # Tomar el (n-1)-grama
    prefix = tuple(text.split()[-(n-1):])
    
    # Obtener las posibles siguientes palabras y sus frecuencias
    next_word_candidates = model[prefix]
    
    # Devolver la palabra con la frecuencia más alta
    if next_word_candidates:
        return next_word_candidates.most_common(1)[0][0]
    else:
        return None  # No hay palabras posibles

In [9]:
# Función para evaluar el desempeño predictivo del modelo con base en un archivo del corpus
def evaluar(text, model, n):
    """
    Evalúa el modelo de n-gramas y muestra las predicciones realizadas.
    
    text: Texto a evaluar
    model: Modelo n-grama entrenado
    n: Tamaño del n-grama
    """

    # Tokenización del texto
    tokens = nltk.word_tokenize(text.lower())
    
    # Variables de conteo
    correct_predictions = 0
    total_predictions = 0
    
    # Obtener los n-gramas del texto que servirán de referencia
    for i in range(len(tokens) - n):
        prefix_tokens = tokens[i:i+(n-1)]
        prefix = "-".join(f"'{token}'" for token in prefix_tokens)
        true_word = tokens[i+(n-1)]
        predicted_word = predict_next_word(model, " ".join(prefix_tokens), n)
        
        # Mostrar la predicción y el resultado esperado
        if predicted_word is None:
            print(f"Predicción para {prefix}: SIN PREDICCIÓN. Debería ser: '{true_word}'")
        else:
            print(f"Predicción para {prefix}: '{predicted_word}'. Debería ser: '{true_word}'")
        
        # Evaluar predicción
        if predicted_word == true_word:
            correct_predictions += 1
        total_predictions += 1
    
    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    print(f"\nAciertos: {correct_predictions}/{total_predictions} = {accuracy * 100:.2f}%\n")

In [10]:
# Ejemplo con un archivo del corpus Reuters
text = reuters.raw('test/14826')

print("2-grama con signos de puntuación:\n")
evaluar(text, bigram_model_with_punctuation, 2)

2-grama con signos de puntuación:

Predicción para 'asian': 'nations'. Debería ser: 'exporters'
Predicción para 'exporters': 'report'. Debería ser: 'fear'
Predicción para 'fear': 'of'. Debería ser: 'damage'
Predicción para 'damage': 'to'. Debería ser: 'from'
Predicción para 'from': 'the'. Debería ser: 'u.s.-japan'
Predicción para 'u.s.-japan': 'trade'. Debería ser: 'rift'
Predicción para 'rift': 'mounting'. Debería ser: 'mounting'
Predicción para 'mounting': 'trade'. Debería ser: 'trade'
Predicción para 'trade': 'deficit'. Debería ser: 'friction'
Predicción para 'friction': 'materials'. Debería ser: 'between'
Predicción para 'between': 'the'. Debería ser: 'the'
Predicción para 'the': 'company'. Debería ser: 'u.s.'
Predicción para 'u.s.': 'agriculture'. Debería ser: 'and'
Predicción para 'and': 'the'. Debería ser: 'japan'
Predicción para 'japan': ''s'. Debería ser: 'has'
Predicción para 'has': 'been'. Debería ser: 'raised'
Predicción para 'raised': 'its'. Debería ser: 'fears'
Predicción

In [11]:
print("2-grama sin signos de puntuación:\n")
evaluar(text, bigram_model_without_punctuation, 2)

2-grama sin signos de puntuación:

Predicción para 'asian': 'nations'. Debería ser: 'exporters'
Predicción para 'exporters': 'report'. Debería ser: 'fear'
Predicción para 'fear': 'of'. Debería ser: 'damage'
Predicción para 'damage': 'to'. Debería ser: 'from'
Predicción para 'from': 'the'. Debería ser: 'u.s.-japan'
Predicción para 'u.s.-japan': 'trade'. Debería ser: 'rift'
Predicción para 'rift': 'mounting'. Debería ser: 'mounting'
Predicción para 'mounting': 'trade'. Debería ser: 'trade'
Predicción para 'trade': 'deficit'. Debería ser: 'friction'
Predicción para 'friction': 'materials'. Debería ser: 'between'
Predicción para 'between': 'the'. Debería ser: 'the'
Predicción para 'the': 'company'. Debería ser: 'u.s.'
Predicción para 'u.s.': 'agriculture'. Debería ser: 'and'
Predicción para 'and': 'the'. Debería ser: 'japan'
Predicción para 'japan': ''s'. Debería ser: 'has'
Predicción para 'has': 'been'. Debería ser: 'raised'
Predicción para 'raised': 'its'. Debería ser: 'fears'
Predicción

In [12]:
print("4-grama con signos de puntuación:\n")
evaluar(text, bigram_model_with_punctuation, 4)

4-grama con signos de puntuación:

Predicción para 'asian'-'exporters'-'fear': SIN PREDICCIÓN. Debería ser: 'damage'
Predicción para 'exporters'-'fear'-'damage': SIN PREDICCIÓN. Debería ser: 'from'
Predicción para 'fear'-'damage'-'from': SIN PREDICCIÓN. Debería ser: 'u.s.-japan'
Predicción para 'damage'-'from'-'u.s.-japan': SIN PREDICCIÓN. Debería ser: 'rift'
Predicción para 'from'-'u.s.-japan'-'rift': SIN PREDICCIÓN. Debería ser: 'mounting'
Predicción para 'u.s.-japan'-'rift'-'mounting': SIN PREDICCIÓN. Debería ser: 'trade'
Predicción para 'rift'-'mounting'-'trade': SIN PREDICCIÓN. Debería ser: 'friction'
Predicción para 'mounting'-'trade'-'friction': SIN PREDICCIÓN. Debería ser: 'between'
Predicción para 'trade'-'friction'-'between': SIN PREDICCIÓN. Debería ser: 'the'
Predicción para 'friction'-'between'-'the': SIN PREDICCIÓN. Debería ser: 'u.s.'
Predicción para 'between'-'the'-'u.s.': SIN PREDICCIÓN. Debería ser: 'and'
Predicción para 'the'-'u.s.'-'and': SIN PREDICCIÓN. Debería ser:

In [13]:
print("4-grama sin signos de puntuación:\n")
evaluar(text, bigram_model_without_punctuation, 4)

4-grama sin signos de puntuación:

Predicción para 'asian'-'exporters'-'fear': SIN PREDICCIÓN. Debería ser: 'damage'
Predicción para 'exporters'-'fear'-'damage': SIN PREDICCIÓN. Debería ser: 'from'
Predicción para 'fear'-'damage'-'from': SIN PREDICCIÓN. Debería ser: 'u.s.-japan'
Predicción para 'damage'-'from'-'u.s.-japan': SIN PREDICCIÓN. Debería ser: 'rift'
Predicción para 'from'-'u.s.-japan'-'rift': SIN PREDICCIÓN. Debería ser: 'mounting'
Predicción para 'u.s.-japan'-'rift'-'mounting': SIN PREDICCIÓN. Debería ser: 'trade'
Predicción para 'rift'-'mounting'-'trade': SIN PREDICCIÓN. Debería ser: 'friction'
Predicción para 'mounting'-'trade'-'friction': SIN PREDICCIÓN. Debería ser: 'between'
Predicción para 'trade'-'friction'-'between': SIN PREDICCIÓN. Debería ser: 'the'
Predicción para 'friction'-'between'-'the': SIN PREDICCIÓN. Debería ser: 'u.s.'
Predicción para 'between'-'the'-'u.s.': SIN PREDICCIÓN. Debería ser: 'and'
Predicción para 'the'-'u.s.'-'and': SIN PREDICCIÓN. Debería ser:

In [14]:
print("2-skip-2-grama con signos de puntuación:\n")
evaluar(text, skip_2_gram_model_with_punctuation, 2)

2-skip-2-grama con signos de puntuación:

Predicción para 'asian': 'the'. Debería ser: 'exporters'
Predicción para 'exporters': 'tonnes'. Debería ser: 'fear'
Predicción para 'fear': 'will'. Debería ser: 'damage'
Predicción para 'damage': 'pipeline'. Debería ser: 'from'
Predicción para 'from': 'in'. Debería ser: 'u.s.-japan'
Predicción para 'u.s.-japan': 'trade'. Debería ser: 'rift'
Predicción para 'rift': 'friction'. Debería ser: 'mounting'
Predicción para 'mounting': ','. Debería ser: 'trade'
Predicción para 'trade': 'the'. Debería ser: 'friction'
Predicción para 'friction': 'two'. Debería ser: 'between'
Predicción para 'between': 'and'. Debería ser: 'the'
Predicción para 'the': 'the'. Debería ser: 'u.s.'
Predicción para 'u.s.': 'the'. Debería ser: 'and'
Predicción para 'and': ','. Debería ser: 'japan'
Predicción para 'japan': 'to'. Debería ser: 'has'
Predicción para 'has': 'to'. Debería ser: 'raised'
Predicción para 'raised': 'in'. Debería ser: 'fears'
Predicción para 'fears': 'infla

In [15]:
print("2-skip-2-grama sin signos de puntuación:\n")
evaluar(text, skip_2_gram_model_without_punctuation, 2)

2-skip-2-grama sin signos de puntuación:

Predicción para 'asian': 'the'. Debería ser: 'exporters'
Predicción para 'exporters': 'tonnes'. Debería ser: 'fear'
Predicción para 'fear': 'will'. Debería ser: 'damage'
Predicción para 'damage': 'pipeline'. Debería ser: 'from'
Predicción para 'from': 'in'. Debería ser: 'u.s.-japan'
Predicción para 'u.s.-japan': 'trade'. Debería ser: 'rift'
Predicción para 'rift': 'friction'. Debería ser: 'mounting'
Predicción para 'mounting': 'to'. Debería ser: 'trade'
Predicción para 'trade': 'the'. Debería ser: 'friction'
Predicción para 'friction': 'the'. Debería ser: 'between'
Predicción para 'between': 'and'. Debería ser: 'the'
Predicción para 'the': 'the'. Debería ser: 'u.s.'
Predicción para 'u.s.': 'the'. Debería ser: 'and'
Predicción para 'and': 'the'. Debería ser: 'japan'
Predicción para 'japan': 'to'. Debería ser: 'has'
Predicción para 'has': 'to'. Debería ser: 'raised'
Predicción para 'raised': 'in'. Debería ser: 'fears'
Predicción para 'fears': 'in

* **Realizar una generación de texto comenzando en dos diferentes *seeds*.**

In [16]:
import numpy as np

# Función para que los modelos generen texto de forma automática
def generar_texto(seed, model, n, total_len=25):
    """
    Genera texto a partir de un seed dado utilizando un modelo entrenado.
    
    :param seed: Semilla inicial de n-1 palabras (tupla)
    :param model: Modelo entrenado de n-gramas
    :param n: Tamaño del n-grama
    :param total_len: Longitud total del texto a generar (número de palabras)
    """
    word_tuple = seed
    pos = 0
    
    # Imprimir el seed
    print(" ".join(word_tuple), end=' ')
    
    # Generar texto
    while pos < total_len:
        next_word = predict_next_word(model, " ".join(word_tuple), n)
        
        if not next_word:
            break  # Si no hay predicción, detener la generación de texto
        
        print(next_word, end=' ')
        word_tuple = word_tuple[1:] + (next_word,)  # Actualizar el tuple de palabras
        pos += 1
    
    print("\n")  # Salto de línea al final del texto generado

In [17]:
# Modelo 2-grama con signos de puntuación (semilla 1)
print("Texto generado con 2-grama con signos de puntuación (semilla 1):")
seed_bigram = list(bigram_model_with_punctuation.keys())[90][:1]  # Obtener 1 palabra del propio vocabulario del modelo
generar_texto(seed_bigram, bigram_model_with_punctuation, 2)

# Modelo 2-grama con signos de puntuación (semilla 2)
print("Texto generado con 2-grama con signos de puntuación (semilla 2):")
seed_bigram = list(bigram_model_with_punctuation.keys())[94][:1]  # Obtener 1 palabra del propio vocabulario del modelo
generar_texto(seed_bigram, bigram_model_with_punctuation, 2)

Texto generado con 2-grama con signos de puntuación (semilla 1):
semiconductors . the company said . the company said . the company said . the company said . the company said . the company said . 

Texto generado con 2-grama con signos de puntuación (semilla 2):
below the company said . the company said . the company said . the company said . the company said . the company said . the 



In [18]:
# Modelo 2-grama sin signos de puntuación (semilla 1)
print("Texto generado con 2-grama sin signos de puntuación (semilla 1):")
seed_bigram = list(bigram_model_without_punctuation.keys())[90][:1]  # Obtener 1 palabra del propio vocabulario del modelo
generar_texto(seed_bigram, bigram_model_with_punctuation, 2)

# Modelo 2-grama sin signos de puntuación (semilla 2)
print("Texto generado con 2-grama sin signos de puntuación (semilla 2):")
seed_bigram = list(bigram_model_without_punctuation.keys())[94][:1]  # Obtener 1 palabra del propio vocabulario del modelo
generar_texto(seed_bigram, bigram_model_with_punctuation, 2)

Texto generado con 2-grama sin signos de puntuación (semilla 1):
markets . the company said . the company said . the company said . the company said . the company said . the company said . 

Texto generado con 2-grama sin signos de puntuación (semilla 2):
unofficial targets for the company said . the company said . the company said . the company said . the company said . the company said 



In [19]:
# Modelo 4-grama con signos de puntuación (semilla 1)
print("Texto generado con 4-grama con signos de puntuación (semilla 1):")
seed_fourgram = list(fourgram_model_with_punctuation.keys())[90][:3]  # Obtener 3 palabras del propio vocabulario del modelo
generar_texto(seed_fourgram, fourgram_model_with_punctuation, 4)

# Modelo 4-grama con signos de puntuación (semilla 2)
print("Texto generado con 4-grama con signos de puntuación (semilla 2):")
seed_fourgram = list(fourgram_model_with_punctuation.keys())[94][:3]  # Obtener 3 palabras del propio vocabulario del modelo
generar_texto(seed_fourgram, fourgram_model_with_punctuation, 4)

Texto generado con 4-grama con signos de puntuación (semilla 1):
be their gain . the u.s. has been pressing west germany to stimulate economic growth and therefore an expansion of the cape spencer deposit had 200,000 tons . 

Texto generado con 4-grama con signos de puntuación (semilla 2):
the u.s. has been pressing west germany to stimulate economic growth and therefore an expansion of the cape spencer deposit had 200,000 tons . gordex said the expansion 



In [20]:
# Modelo 4-grama sin signos de puntuación (semilla 1)
print("Texto generado sin 4-grama con signos de puntuación (semilla 1):")
seed_fourgram = list(fourgram_model_without_punctuation.keys())[90][:3]  # Obtener 3 palabras del propio vocabulario del modelo
generar_texto(seed_fourgram, fourgram_model_with_punctuation, 4)

# Modelo 4-grama sin signos de puntuación (semilla 2)
print("Texto generado sin 4-grama con signos de puntuación (semilla 2):")
seed_fourgram = list(fourgram_model_without_punctuation.keys())[94][:3]  # Obtener 3 palabras del propio vocabulario del modelo
generar_texto(seed_fourgram, fourgram_model_with_punctuation, 4)


Texto generado sin 4-grama con signos de puntuación (semilla 1):
u.s. has said it is in talks with the world bank and the international monetary fund ( imf ) to reduce the deficit , such as the sharp 

Texto generado sin 4-grama con signos de puntuación (semilla 2):
will impose 300 mln dlrs in the first quarter of 1987 , the company said . the company said . the company said . the company said . 



In [21]:
# Modelo 2-skip-2-grama con signos de puntuación (semilla 1)
print("Texto generado con 2-skip-2-grama con signos de puntuación:")
seed_skip_2_gram = list(skip_2_gram_model_with_punctuation.keys())[90][:1]  # Obtener 1 palabra del propio vocabulario del modelo
generar_texto(seed_skip_2_gram, skip_2_gram_model_with_punctuation, 2)

# Modelo 2-skip-2-grama con signos de puntuación (semilla 1)
print("Texto generado con 2-skip-2-grama con signos de puntuación:")
seed_skip_2_gram = list(skip_2_gram_model_with_punctuation.keys())[94][:1]  # Obtener 1 palabra del propio vocabulario del modelo
generar_texto(seed_skip_2_gram, skip_2_gram_model_with_punctuation, 2)

Texto generado con 2-skip-2-grama con signos de puntuación:
official the the the the the the the the the the the the the the the the the the the the the the the the the 

Texto generado con 2-skip-2-grama con signos de puntuación:
also the the the the the the the the the the the the the the the the the the the the the the the the the 



In [22]:
# Modelo 2-skip-2-grama sin signos de puntuación (semilla 1)
print("Texto generado sin 2-skip-2-grama con signos de puntuación:")
seed_skip_2_gram = list(skip_2_gram_model_without_punctuation.keys())[90][:1]  # Obtener 1 palabra del propio vocabulario del modelo
generar_texto(seed_skip_2_gram, skip_2_gram_model_with_punctuation, 2)

# Modelo 2-skip-2-grama sin signos de puntuación (semilla 1)
print("Texto generado con 2-skip-2-grama sin signos de puntuación:")
seed_skip_2_gram = list(skip_2_gram_model_without_punctuation.keys())[94][:1]  # Obtener 1 palabra del propio vocabulario del modelo
generar_texto(seed_skip_2_gram, skip_2_gram_model_with_punctuation, 2)

Texto generado sin 2-skip-2-grama con signos de puntuación:
pressure to , said the the the the the the the the the the the the the the the the the the the the the the 

Texto generado con 2-skip-2-grama sin signos de puntuación:
malaysia , said the the the the the the the the the the the the the the the the the the the the the the the 



⭕ Discutir y redactar un reporte (en el mismo notebook) abordando los siguientes puntos:

**1. ¿Qué modelo de los 6 logró mejores predicciones?**

En términos de precisión, los modelos de bigramas (2-gramas) con y sin signos de puntuación lograron las mejores predicciones, aunque sus tasas de aciertos fueron relativamente bajas (alrededor del 20.77% y 18.67%, respectivamente). Esto sugiere que, a pesar de las limitaciones inherentes a los bigramas, estos modelos fueron los más efectivos en capturar relaciones locales entre palabras en el corpus utilizado.

**2. ¿Qué modelo de los 6 tenía más problemas generando predicciones?**

Los modelos de 4-gramas, tanto con como sin signos de puntuación, presentaron serios problemas en la generación de predicciones, con una tasa de aciertos del 0%. Este comportamiento indica que los 4-gramas no fueron capaces de encontrar coincidencias adecuadas en el texto de prueba, lo que podría deberse a la falta de suficiente cobertura de los n-gramas en el corpus de entrenamiento o a una sobreespecificidad de los modelos que no pudo generalizar bien en el texto de prueba.

**3. ¿Qué efecto tiene el tipo de corpus en la construcción y desempeño del modelo?**

El tipo de corpus tiene un impacto significativo en la construcción y desempeño de los modelos de n-gramas. El corpus Reuters, utilizado en este ejercicio, es un conjunto de noticias financieras con un lenguaje específico y ciertas frases repetitivas, como "the company said". Esto explica por qué los bigramas tuvieron un rendimiento relativamente mejor: pudieron capturar estas frases repetidas. Sin embargo, la especificidad del corpus también significa que los modelos más complejos como los 4-gramas o los skip-gramas pueden carecer de suficiente información contextual en el entrenamiento para hacer predicciones precisas. Además, la repetitividad en el corpus puede llevar a que los modelos generen texto que simplemente repite las frases más comunes sin aportar nueva información.

**4. ¿Qué podríamos hacer para incrementar los aciertos?**

Para mejorar el rendimiento de los modelos de n-gramas, podríamos considerar varias estrategias:

* **Aumentar el tamaño del corpus:** Un corpus más grande y diverso podría proporcionar una mejor cobertura de los posibles n-gramas y reducir la especificidad excesiva, permitiendo que los modelos generalicen mejor.

* **Mejorar la preprocesamiento del texto:** Limpiar y normalizar el texto de manera más efectiva, como la eliminación de palabras muy comunes o el manejo de sinónimos, podría ayudar a mejorar la calidad de las predicciones.

* **Incorporar técnicas de suavizado:** Técnicas como el suavizado de Laplace pueden ayudar a lidiar con el problema de la falta de cobertura en el modelo, asignando una probabilidad mínima a n-gramas no vistos.